Implement fuzzy matching on LLCs that have filed evictions to lookup registered LLC info

Updated 4/8/25

In [3]:
import pandas as pd
import numpy as np
from concurrent.futures import ProcessPoolExecutor
from rapidfuzz import process, fuzz
import io

In [7]:
a = pd.read_csv('DATA/llc.csv')

/var/folders/ll/gtcbtj4n6mjf4mwvq3711yw40000gn/T/ipykernel_2688/1095993889.py:1: DtypeWarning: Columns (0,23) have mixed types. Specify dtype option on import or set low_memory=False.
  a = pd.read_csv('DATA/llc.csv')


In [5]:
df = pd.read_csv("DATA/llc.csv", 
                 delimiter=",", 
                 skipinitialspace=True,  # Skip spaces after delimiter
                 quotechar='"',          # Properly handle quoted fields
                 dtype=str)              # Load all columns as strings initially

# Clean up column names
df.columns = df.columns.str.strip()

# If the first column is empty or just contains indices
if df.columns[0] == 'Unnamed: 0' or df.columns[0].strip() == '':
    df = df.iloc[:, 1:]  # Remove the first column

# Further cleanup if needed
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

# Display the cleaned dataframe
print(df.head())

/var/folders/ll/gtcbtj4n6mjf4mwvq3711yw40000gn/T/ipykernel_2688/2962749221.py:15: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)


                                                    EntityID      Name  \
\t11072857                                TJ PROPERTIES, LLC  INACTIVE   
\t11159320    Rehab Developrs & Management House Buyers, LLC  INACTIVE   
\t11468309               SIMPLISTIC HOLISTIC PHOTOGRAPHY LLC  INACTIVE   
\t00601153                            Handy Distribution LLC    ACTIVE   
\t00036251                  Lumos Telephone of Botetourt LLC    ACTIVE   

                Status StatusReason Status Date    Duration IncorpDate  \
\t11072857    Rejected   2020-07-06  9999-12-31         NaN         VA   
\t11159320    Rejected   2021-01-13  9999-12-31         NaN         VA   
\t11468309    Rejected   2022-12-07  9999-12-31         NaN         VA   
\t00601153      Active   2006-03-17  9999-12-31  1899-02-16         VA   
\t00036251      Active   2019-04-26  9999-12-31  1901-02-23         VA   

                 IncorpState IndustryCode Street1  ... RA-EffDate RA-Status  \
\t11072857               NaN   

In [6]:
df.columns

Index(['EntityID', 'Name', 'Status', 'StatusReason', 'Status Date', 'Duration',
       'IncorpDate', 'IncorpState', 'IndustryCode', 'Street1', 'Street2',
       'City', 'State', 'Zip', 'PrinOffEffDate', 'RA-Name', 'RA-Street1',
       'RA-Street2', 'RA-City', 'RA-State', 'RA-Zip', 'RA-EffDate',
       'RA-Status', 'RA-Loc', 'StockInd', 'TotalShares', 'MergerInd',
       'AssessInd', 'Is Series LLC', 'Is Protected Series', 'Series LLC ID'],
      dtype='object')

In [3]:
evictions_df = pd.read_parquet('evictions.parquet')

In [4]:
llc_df = pd.read_parquet('llc.parquet')

Conservative pre-processing approach to maintain efficiency

Can make this more aggressive if it doesn't work out

In [5]:
def preprocess_names(df, column_name):
    """
    Normalize names, sort words alphabetically for natural language processing if we need it later
    @df: pandas dataframe
    @column_name: str column name to preprocess
    
    """
    column = column_name + "_normalized"
    df[column] = (
        df[column_name]
        .str.lower()
        .str.replace(r'[^a-zA-Z0-9\s]', '', regex=True)
        .str.replace(r'\s+', ' ', regex=True)
        .str.strip()
    ) 

    df[column] = (
        df[column]
        .str.split()
        .map(lambda x: " ".join(sorted(x)) if isinstance(x, list) else ' ')
    )

    return df

In [6]:
e_columns_for_p = ['plaintiff_name'] # columns for preprocessing, add more as needed
l_columns_for_p = ['EntityID']

In [7]:
for i in e_columns_for_p: # apply preprocessing for both df
    evictions_df = preprocess_names(evictions_df, i)

for i in l_columns_for_p:
    llc_df = preprocess_names(llc_df, i)

In [8]:
# create flag 'is_llc' to restrict lookup to only llcs
llc_keywords = r"\b(llc|l\.l\.c|inc|inc\.|corporation|corp|corp\.|co|co\.|company|ltd|ltd\.|lp|l\.p\.|pllc|plc|plc\.|limited|limited liability company)\b"
evictions_df["is_llc"] = evictions_df["plaintiff_name_normalized"].str.contains(llc_keywords, regex=True)

/tmp/ipykernel_229293/4089579124.py:3: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  evictions_df["is_llc"] = evictions_df["plaintiff_name_normalized"].str.contains(llc_keywords, regex=True)


In [9]:
# Now that we've set the 'is_llc' flag, we can remove these terms to reduce noise
suffixes = [
    "llc", "l.l.c", "inc", "incorporated", "corp", "corporation",
    "co", "company", "ltd", "l.p", "lp", "pllc", "plc", "llp", "p.c"
]

suffix_pattern = r'\s*(?:' + '|'.join(suffixes) + r')\.?\s*$'

evictions_df['plaintiff_name_normalized'] = evictions_df['plaintiff_name_normalized'].str.replace(suffix_pattern, '', regex=True)
llc_df['EntityID_normalized'] = llc_df['EntityID_normalized'].str.replace(suffix_pattern, '', regex=True)

In [14]:
evictions_df = evictions_df[evictions_df['is_llc']] # restrict lookup to only llcs
plaintiffs = set(evictions_df['plaintiff_name_normalized']) # set of unique LLC plaintiffs
llcs = list(set(llc_df['EntityID_normalized'])) # set of unique registered LLCs

In [18]:
# Function to process matching for a subset of plaintiff names
def process_chunk(plaintiff_chunk, llc_names, confidence=80):
    matches = {}
    for i, plaintiff in enumerate(plaintiff_chunk):
        best_match, score, _ = process.extractOne(plaintiff, llc_names, scorer=fuzz.WRatio)
        # if best_match and score >= confidence:
        matches[plaintiff] = (best_match, score)
        # else:
            # matches[plaintiff] = (None, 0)
        if (i + 1) % 100 == 0:
            print(f"Processed {i + 1} out of {len(plaintiff_chunk)} plaintiffs in this chunk.")
    return matches

# Function to split the plaintiff set and process in parallel
def parallel_match_plaintiffs(plaintiff_names_set, llc_names_set, chunk_size=1000):
    plaintiff_chunks = [list(plaintiff_names_set)[i:i + chunk_size] for i in range(0, len(plaintiff_names_set), chunk_size)]
    print(f"Total of {len(plaintiff_chunks)} chunks to process.")

    all_matches = {}
    with ProcessPoolExecutor() as executor:
        futures = [executor.submit(process_chunk, chunk, llc_names_set) for chunk in plaintiff_chunks]
        for i, future in enumerate(futures):
            all_matches.update(future.result())
            print(f"Chunk {i + 1} processed. Total matches found: {len(all_matches)}")
    
    return all_matches

In [16]:
chunk_size = len(plaintiffs) // 16 # adjust for num of cores, 16 worked well under an hour
# probably only needed ~100GB memory or less

In [19]:
# Matches looks like:
# {plaintiff_name: (best_match_llc, confidence_score)}
matches = parallel_match_plaintiffs(plaintiffs, llcs, chunk_size=chunk_size) 

Total of 17 chunks to process.
Processed 100 out of 972 plaintiffs in this chunk.
Processed 100 out of 972 plaintiffs in this chunk.
Processed 100 out of 972 plaintiffs in this chunk.
Processed 100 out of 972 plaintiffs in this chunk.
Processed 100 out of 972 plaintiffs in this chunk.
Processed 100 out of 972 plaintiffs in this chunk.
Processed 100 out of 972 plaintiffs in this chunk.
Processed 100 out of 972 plaintiffs in this chunk.
Processed 100 out of 972 plaintiffs in this chunk.
Processed 100 out of 972 plaintiffs in this chunk.
Processed 100 out of 972 plaintiffs in this chunk.
Processed 100 out of 972 plaintiffs in this chunk.
Processed 100 out of 972 plaintiffs in this chunk.
Processed 100 out of 972 plaintiffs in this chunk.
Processed 100 out of 972 plaintiffs in this chunk.
Processed 100 out of 972 plaintiffs in this chunk.
Processed 200 out of 972 plaintiffs in this chunk.
Processed 200 out of 972 plaintiffs in this chunk.
Processed 200 out of 972 plaintiffs in this chunk.


In [20]:
evictions_df['match_tuple'] = evictions_df['plaintiff_name_normalized'].map(matches) # Match tuple looks like (best_match, confidence_score)
# Unpack into two separate columns
evictions_df[['best_match', 'match_confidence']] = pd.DataFrame(evictions_df['match_tuple'].tolist(), index=evictions_df.index)

In [21]:
evictions_df = pd.merge(
    left=evictions_df,
    right=llc_df,
    left_on='best_match',
    right_on='EntityID_normalized',
    how='left'
).drop(columns=['best_match', 'match_tuple'])

In [25]:
evictions_df.to_parquet('evictions_matched.parquet')